In [ ]:
import cv2
import numpy as np 
import tensorflow as tf 
import tflite_runtime.interpreter as tflite
import time

tf.__version__

In [ ]:
# 加载TensorflowLite模型
interpreter = tf.lite.Interpreter(model_path='hand_landmark.tflite')
interpreter.allocate_tensors()

# 输入输出信息参考ModelCard
input_details = interpreter.get_input_details()
print(str(input_details))
output_details = interpreter.get_output_details()
print(str(output_details))

In [ ]:
# 使用1张图片测试
img_path='test.jpg'

img=cv2.imread(img_path)
img = cv2.cvtColor(cv2.flip(img, 1), cv2.COLOR_BGR2RGB)
img=cv2.resize(img,(224,224))

img_array=np.array(img)
img_array=img_array.astype('float32')
img_array/=255
img_array=np.expand_dims(img_array,axis=0)

print(img_array.shape)

interpreter.set_tensor(input_details[0]['index'], img_array)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

print(output_data)


# 在图片中添加骨骼点
for i in range(21):
    x=int(output_data[0][3*i])
    y=int(output_data[0][3*i+1])
    cor=(x,y)
    cv2.circle(img,cor,1,(0,0,255),3)

# 转换为BGR输出
img = cv2.cvtColor(cv2.flip(img, 1), cv2.COLOR_RGB2BGR)
cv2.imwrite('test_hand.jpeg', img)


In [ ]:
def Show_frame(img,cors):
    for i in range(21):
        cor=(int(cors[3*i]),int(cors[3*i+1]))
        cv2.circle(img,cor,1,(0,0,255),3)


In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():

  t_start=time.time()

  success, img = cap.read()
  if not success:
    print("Ignoring empty camera frame.")
    continue
  img = cv2.cvtColor(cv2.flip(img, 1), cv2.COLOR_BGR2RGB)

  img=cv2.resize(img,(224,224))
  
  img_array=np.array(img)
  img_array=img_array.astype('float32')
  img_array/=255
  img_array=np.expand_dims(img_array,axis=0)

  #print(img_array.shape)
  
  interpreter.set_tensor(input_details[0]['index'], img_array)
  interpreter.invoke()
  output_data = interpreter.get_tensor(output_details[0]['index'])
  confidence = interpreter.get_tensor(output_details[1]['index'])
  LR=interpreter.get_tensor(output_details[2]['index'])

  Show_frame(img,output_data[0])

  img=cv2.resize(img,(640,480))
  img = cv2.cvtColor(cv2.flip(img, 1), cv2.COLOR_RGB2BGR)
  cv2.imshow('MediaPipe Hands', img)

  if cv2.waitKey(100) & 0xFF == 27: # 按下 esc 退出
    break

  dt=time.time()-t_start
  print('FPS: ',1/dt,'confidence:',confidence,'L or R:',LR)
cap.release()

In [ ]:
cap = cv2.VideoCapture(0)

while cap.isOpened():

  t_start=time.time()

  success, img = cap.read()
  if not success:
    print("Ignoring empty camera frame.")
    continue

  print(img.shape)
  cv2.imshow('MediaPipe Hands', img)

  if cv2.waitKey(100) & 0xFF == 27: # 按下 esc 退出
    break

  dt=time.time()-t_start
cap.release()